# Classification using Deep Neural Network

## Import Native and Third Party Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import tensorflow as tf

/home/abhishek/Desktop/Projects/tf/venv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read Dataset

In [2]:
os.chdir('/home/abhishek/Desktop/Projects/tf/yet_another_ML_tutorial/coding_exercise/')
raw_data = pd.read_csv("./CreditDataset.csv", header=None)
print "Shape of original data frame:", raw_data.shape

Shape of original data frame: (1000, 21)


In [3]:
# Get data types
print raw_data.dtypes
obj_df = raw_data.select_dtypes(include=['object']).copy()
print "Shape of object data frame:", obj_df.shape
int_df = raw_data.select_dtypes(include=['int64']).copy()
print "Shape of int64 data frame:", int_df.shape
print "Type of int data frame:", type(int_df)

# Check for null values in the columns containing categorical variables
print obj_df[obj_df.isnull().any(axis=1)]

0     object
1      int64
2     object
3     object
4      int64
5     object
6     object
7      int64
8     object
9     object
10     int64
11    object
12     int64
13    object
14    object
15     int64
16    object
17     int64
18    object
19    object
20     int64
dtype: object
Shape of object data frame: (1000, 13)
Shape of int64 data frame: (1000, 8)
Type of int data frame: <class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
Index: []


## Data Preprocessing
Categorical data has been converted into numerical data using `one-hot encoding` in `scikit-learn`. Output labels have been one-hot encoded using `numpy`. The resulting dataset has the shape `(1000,63)`. 

In [4]:
# One hot encoding of the columns containing categorical variables
# Label encoder
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
# FIT AND TRANSFORM. use df.apply() to apply le.fit_transform to all columns
le_obj_df = obj_df.apply(le.fit_transform)
# print raw_data.select_dtypes(include=['object']).head(5)
# print le_obj_df.head()

# One hot encoding of categorical variables
# 1. INSTANTIATE
encode_object = preprocessing.OneHotEncoder()
# 2. FIT
encode_object.fit(le_obj_df)
# 3. Transform
onehotlabels = encode_object.transform(le_obj_df).toarray()
print onehotlabels.shape
print type(onehotlabels)

# Merge the int64 data frame with the one hot labels
np_int_df = int_df.as_matrix()
print np_int_df.shape
processed_data = np.concatenate([onehotlabels, np_int_df], axis=1)
print processed_data.shape

# print processed_data[:,-1]
print processed_data.dtype


# Encode output for NN
# encode_output = preprocessing.OneHotEncoder()
# encode_output.fit(processed_data[:,-1])
# output_onehotlabels = encode_output.transform(processed_data[:,-1])
# print "Shape of encoded one hot labels:", output_onehotlabels.shape
#

raw_labels = np.array(processed_data[:,-1]).astype(int)
encoded_labels = np.zeros((processed_data[:,-1].shape[0], 2))
encoded_labels[np.arange(processed_data[:,-1].shape[0]), raw_labels-1] = 1


processed_data = processed_data[:,0:61]
processed_data = np.concatenate([processed_data, encoded_labels], axis=1)
print processed_data.shape

(1000, 54)
<type 'numpy.ndarray'>
(1000, 8)
(1000, 62)
float64
(1000, 63)


## Data Split
30% of the actual data has been kept as test data.  

In [5]:
# Get test train split
X_train, X_test, y_train, y_test = train_test_split(processed_data[:, 0:61],
                                                    processed_data[:, 61:63],
                                                    test_size=0.3,
                                                    random_state=42)


In [6]:
print X_train.shape, X_test.shape

(700, 61) (300, 61)


In [7]:
print y_train.shape, y_test.shape

(700, 2) (300, 2)


## Mini-batch Creation
At each iteration of training, a mini-batch of the data shall be fed to the network. 

In [8]:
def random_batch(dataset, batch_size):
    sample = dataset[np.random.choice(dataset.shape[0], batch_size, replace=False),:]
    last_col_index = dataset.shape[1]-2
    x = sample[:,0:last_col_index]
    y = sample[:,last_col_index:last_col_index+2]
    return (x, y)

## Neural Network
Two hidden layer each with 256 neurons. Network was trained with `Adam Optimizer`. Network parameters are mentioned in the snippet below:

```python
learning_rate = 0.001
num_steps = 50000
batch_size = 100
display_step = 100
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
```

In [9]:
# Parameters
learning_rate = 0.001
num_steps = 50000
batch_size = 100
display_step = 100
weighted_cost = True # Apply weight of 5 and 1 for FPs and TNs respectively
false_neg_cost = 5.0
false_pos_cost = 1.0

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = X_train.shape[1] 
num_classes = y_train.shape[1]

print "Number of input:", num_input
print "Number of classes:", num_classes

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

Number of input: 61
Number of classes: 2


In [10]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [11]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [12]:
# Construct model
logits = neural_net(X)

if weighted_cost:
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(
              targets=Y, logits=logits, pos_weight=false_neg_cost))
else:
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = random_batch(np.concatenate([X_train, y_train], axis=1), 
                                        batch_size) 
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 635129.9375, Training Accuracy= 0.730
Step 100, Minibatch Loss= 2885.2468, Training Accuracy= 0.560
Step 200, Minibatch Loss= 14991.1396, Training Accuracy= 0.660
Step 300, Minibatch Loss= 33021.5586, Training Accuracy= 0.320
Step 400, Minibatch Loss= 18281.2617, Training Accuracy= 0.680
Step 500, Minibatch Loss= 10451.6797, Training Accuracy= 0.530
Step 600, Minibatch Loss= 2014.0500, Training Accuracy= 0.710
Step 700, Minibatch Loss= 8452.2109, Training Accuracy= 0.750
Step 800, Minibatch Loss= 17707.2500, Training Accuracy= 0.740
Step 900, Minibatch Loss= 8238.7061, Training Accuracy= 0.540
Step 1000, Minibatch Loss= 19186.8008, Training Accuracy= 0.650
Step 1100, Minibatch Loss= 32687.4941, Training Accuracy= 0.240
Step 1200, Minibatch Loss= 7185.9683, Training Accuracy= 0.550
Step 1300, Minibatch Loss= 6489.5181, Training Accuracy= 0.680
Step 1400, Minibatch Loss= 4641.2939, Training Accuracy= 0.620
Step 1500, Minibatch Loss= 6604.1514, Training Accuracy= 0

Step 13000, Minibatch Loss= 15688.1777, Training Accuracy= 0.480
Step 13100, Minibatch Loss= 8460.1104, Training Accuracy= 0.710
Step 13200, Minibatch Loss= 5064.1221, Training Accuracy= 0.720
Step 13300, Minibatch Loss= 3016.5525, Training Accuracy= 0.760
Step 13400, Minibatch Loss= 2787.6382, Training Accuracy= 0.730
Step 13500, Minibatch Loss= 15361.3975, Training Accuracy= 0.460
Step 13600, Minibatch Loss= 10142.9404, Training Accuracy= 0.590
Step 13700, Minibatch Loss= 18238.1992, Training Accuracy= 0.650
Step 13800, Minibatch Loss= 11876.3799, Training Accuracy= 0.690
Step 13900, Minibatch Loss= 27740.5723, Training Accuracy= 0.420
Step 14000, Minibatch Loss= 4211.9092, Training Accuracy= 0.750
Step 14100, Minibatch Loss= 17109.5605, Training Accuracy= 0.480
Step 14200, Minibatch Loss= 9249.3701, Training Accuracy= 0.700
Step 14300, Minibatch Loss= 4118.9023, Training Accuracy= 0.760
Step 14400, Minibatch Loss= 4394.8154, Training Accuracy= 0.760
Step 14500, Minibatch Loss= 4466.

Step 25800, Minibatch Loss= 11104.0352, Training Accuracy= 0.680
Step 25900, Minibatch Loss= 3177.7637, Training Accuracy= 0.810
Step 26000, Minibatch Loss= 9249.8896, Training Accuracy= 0.730
Step 26100, Minibatch Loss= 4270.5952, Training Accuracy= 0.720
Step 26200, Minibatch Loss= 4268.8574, Training Accuracy= 0.630
Step 26300, Minibatch Loss= 3556.6113, Training Accuracy= 0.770
Step 26400, Minibatch Loss= 7686.4111, Training Accuracy= 0.720
Step 26500, Minibatch Loss= 9107.5430, Training Accuracy= 0.440
Step 26600, Minibatch Loss= 6094.8218, Training Accuracy= 0.580
Step 26700, Minibatch Loss= 11458.1680, Training Accuracy= 0.810
Step 26800, Minibatch Loss= 3160.6426, Training Accuracy= 0.740
Step 26900, Minibatch Loss= 8776.0068, Training Accuracy= 0.800
Step 27000, Minibatch Loss= 2687.4238, Training Accuracy= 0.810
Step 27100, Minibatch Loss= 10534.6123, Training Accuracy= 0.750
Step 27200, Minibatch Loss= 9860.5459, Training Accuracy= 0.720
Step 27300, Minibatch Loss= 17147.619

Step 38600, Minibatch Loss= 8567.9873, Training Accuracy= 0.550
Step 38700, Minibatch Loss= 4804.6401, Training Accuracy= 0.690
Step 38800, Minibatch Loss= 11417.9824, Training Accuracy= 0.670
Step 38900, Minibatch Loss= 7183.2954, Training Accuracy= 0.600
Step 39000, Minibatch Loss= 4263.4678, Training Accuracy= 0.670
Step 39100, Minibatch Loss= 5848.5151, Training Accuracy= 0.750
Step 39200, Minibatch Loss= 10621.1084, Training Accuracy= 0.620
Step 39300, Minibatch Loss= 9774.4189, Training Accuracy= 0.630
Step 39400, Minibatch Loss= 7093.9849, Training Accuracy= 0.810
Step 39500, Minibatch Loss= 3826.5947, Training Accuracy= 0.720
Step 39600, Minibatch Loss= 9021.4863, Training Accuracy= 0.620
Step 39700, Minibatch Loss= 3686.5637, Training Accuracy= 0.650
Step 39800, Minibatch Loss= 2672.4575, Training Accuracy= 0.750
Step 39900, Minibatch Loss= 3135.4463, Training Accuracy= 0.780
Step 40000, Minibatch Loss= 13193.3115, Training Accuracy= 0.680
Step 40100, Minibatch Loss= 23555.679

## Accuracy
Accuracy on the test set after about `50000` iterations was approx. `74%`.